In [ ]:
#!/usr/bin/env python3
# dhs_insurance_sh21_dejure.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math

# ------------------------------------------------------------------
# 1. SETTINGS
# ------------------------------------------------------------------
FILE_PATH = os.path.join("../data", "RWPR81FL.DTA")

# Variable Map
VAR_WT       = "hv005"       # Weight
VAR_SEX      = "hv104"       # Sex (1=Male, 2=Female)
VAR_RESIDENT = "hv102"       # Usual Resident (1=Yes) -> De jure
VAR_REG      = "hv024"       # Region
VAR_DIST     = "shdistrict"  # District

# --- YOUR VARIABLES ---
# sh21: Covered by insurance (1=Yes, 0=No, 8=DK)
# sh22: Main type (used here only for verification context, analysis uses sh21)
VAR_INSURANCE = "sh21"   

DIST_MAP = {11: "Nyarugenge", 12: "Gasabo", 13: "Kicukiro"}
COLORS   = ["#4F81BD", "#C0504D", "#9BBB59"] # Blue (Men), Red (Women), Green (Total)

# ------------------------------------------------------------------
# 2. CALCULATION HELPERS
# ------------------------------------------------------------------
def standard_round(n):
    """0.5 rounds UP to nearest Integer."""
    return int(math.floor(n + 0.5))

def get_pct(df):
    if df.empty or df['w'].sum() == 0:
        return 0
    # Numerator: sh21 == 1 (Yes)
    # Denominator: Everyone in the filtered list (includes 0 and 8)
    val = np.average(df['has_insurance'], weights=df['w']) * 100
    return standard_round(val)

def get_triplet(df_geo):
    """Calculates stats for Men, Women, and Total."""
    # Men: All Ages
    df_men = df_geo[df_geo[VAR_SEX] == 1]

    # Women: All Ages
    df_women = df_geo[df_geo[VAR_SEX] == 2]

    return {
        "Men":   get_pct(df_men),
        "Women": get_pct(df_women),
        "Total": get_pct(df_geo)
    }

# ------------------------------------------------------------------
# 3. ANALYSIS PIPELINE
# ------------------------------------------------------------------
if __name__ == "__main__":
    if not os.path.exists(FILE_PATH):
        print(f"❌ Error: {FILE_PATH} not found.")
        exit()

    print("Loading Data...")
    df = pd.read_stata(FILE_PATH, convert_categoricals=False)
    df.columns = df.columns.str.lower()

    # --- FILTERS (MATCHING FIGURE 8) ---
    
    # 1. De jure Population (Usual Residents)
    # This is the key fix to match the publication numbers (86% vs 83%)
    df = df[df[VAR_RESIDENT] == 1]

    # 2. All Ages
    # We do NOT filter by age. Figure 8 applies to all household members.

    # 3. Weights
    df["w"] = df[VAR_WT] / 1000000.0

    # 4. Indicator (sh21)
    if VAR_INSURANCE in df.columns:
        # 1 = Yes
        # 0 = No, 8 = Don't Know (both count as Not Covered in numerator)
        df["has_insurance"] = (df[VAR_INSURANCE] == 1).astype(int)
    else:
        print(f"❌ Error: Variable {VAR_INSURANCE} not found.")
        exit()

    # --- AGGREGATION ---
    data_rows = {}

    # A. Districts (Kigali)
    df_kigali = df[df[VAR_REG] == 1]
    for dist_code, dist_name in DIST_MAP.items():
        subset = df_kigali[df_kigali[VAR_DIST] == dist_code]
        data_rows[dist_name] = get_triplet(subset)

    # B. Kigali City
    data_rows["Kigali City"] = get_triplet(df_kigali)

    # C. Rwanda
    data_rows["Rwanda"] = get_triplet(df)

    # Convert to DataFrame
    final_df = pd.DataFrame.from_dict(data_rows, orient='index')
    final_df = final_df[["Men", "Women", "Total"]] 
    
    print(f"\n--- Health Insurance Coverage ({VAR_INSURANCE}) ---")
    print("Population: De jure (Usual Residents) | All Ages")
    print(final_df)
    final_df.to_csv("Insurance_sh21_DeJure.csv")

    # ------------------------------------------------------------------
    # 4. PLOTTING
    # ------------------------------------------------------------------
    ax = final_df.plot(kind="bar", color=COLORS, figsize=(14, 7), 
                       width=0.75, edgecolor="white")

    plt.title(f"Health Insurance Coverage (Any Type: {VAR_INSURANCE})\n(De jure Population, All Ages)", 
              fontsize=16, fontweight="bold")
    plt.xticks(rotation=0, fontsize=11)
    plt.grid(axis="y", ls="--", alpha=0.3)
    
    ax.yaxis.set_visible(False)
    for s in ["top", "right", "left"]: ax.spines[s].set_visible(False)

    plt.legend(ncol=3, loc="upper center", bbox_to_anchor=(0.5, -0.08), 
               frameon=False, fontsize=12)

    for c in ax.containers:
        for v in c:
            height = v.get_height()
            label = f"{int(height)}" if height > 0 else ""
            ax.text(v.get_x() + v.get_width() / 2, height + 1, label,
                    ha='center', va='bottom', fontsize=11, fontweight="bold", color="black")

    plt.tight_layout()
    plt.savefig("Insurance_sh21_DeJure.png", dpi=300)
    plt.close()
    print("✅ Analysis Complete.")

In [2]:
#!/usr/bin/env python3
# dhs_insurance_imputed.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math

# ------------------------------------------------------------------
# 1. SETTINGS
# ------------------------------------------------------------------
FILE_PATH = os.path.join("data", "RWPR81FL.DTA")

# Variable Map
VAR_WT       = "hv005"       # Weight
VAR_SEX      = "hv104"       # Sex
VAR_RESIDENT = "hv102"       # Usual Resident (1=Yes) -> De jure
VAR_REG      = "hv024"       # Region
VAR_DIST     = "shdistrict"  # District
VAR_AGE      = "hv105"       # Age

# Household Linking Variables
VAR_CLUSTER  = "hv001"       # Cluster number
VAR_HH_NUM   = "hv002"       # Household number
VAR_RELATION = "hv101"       # Relationship to head (1=Head)

# Your Insurance Variable
VAR_INSURANCE = "sh21"       # 1=Yes, 0=No, 8=DK

DIST_MAP = {11: "Nyarugenge", 12: "Gasabo", 13: "Kicukiro"}
COLORS   = ["#4F81BD", "#C0504D", "#9BBB59"] 

# ------------------------------------------------------------------
# 2. CALCULATION
# ------------------------------------------------------------------
def standard_round(n):
    return int(math.floor(n + 0.5))

def get_pct(df):
    if df.empty or df['w'].sum() == 0:
        return 0
    val = np.average(df['final_coverage'], weights=df['w']) * 100
    return standard_round(val)

def get_triplet(df_geo):
    return {
        "Men":   get_pct(df_geo[df_geo[VAR_SEX] == 1]),
        "Women": get_pct(df_geo[df_geo[VAR_SEX] == 2]),
        "Total": get_pct(df_geo)
    }

# ------------------------------------------------------------------
# 3. ANALYSIS PIPELINE
# ------------------------------------------------------------------
if __name__ == "__main__":
    if not os.path.exists(FILE_PATH):
        print(f"❌ Error: {FILE_PATH} not found.")
        exit()

    print("Loading Data...")
    df = pd.read_stata(FILE_PATH, convert_categoricals=False)
    df.columns = df.columns.str.lower()

    # --- FILTER 1: De jure (Usual Residents) ---
    df = df[df[VAR_RESIDENT] == 1]

    # --- WEIGHTS ---
    df["w"] = df[VAR_WT] / 1000000.0

    # --- BASE COVERAGE (sh21) ---
    # 1=Yes, 0=No, 8=DK.
    # We treat 8 as No (Standard DHS) OR you can drop them.
    # Here we treat 0 and 8 as 0 initially.
    df["raw_insurance"] = (df[VAR_INSURANCE] == 1).astype(int)

    # --------------------------------------------------------------
    # --- CRITICAL FIX: HOUSEHOLD IMPUTATION ---
    # --------------------------------------------------------------
    print("Applying Household Imputation Logic...")
    
    # 1. Isolate Heads of Households (hv101 == 1)
    heads = df[df[VAR_RELATION] == 1].copy()
    
    # 2. Keep only identifying info + insurance status
    heads = heads[[VAR_CLUSTER, VAR_HH_NUM, "raw_insurance"]]
    heads.rename(columns={"raw_insurance": "head_has_ins"}, inplace=True)
    
    # 3. Merge Head's status back to all members
    # We join on Cluster + HH Number
    df = df.merge(heads, on=[VAR_CLUSTER, VAR_HH_NUM], how="left")
    
    # 4. Fill NA for households with no head (rare) with 0
    df["head_has_ins"] = df["head_has_ins"].fillna(0).astype(int)
    
    # 5. Apply Logic:
    # IF Person has insurance -> YES
    # OR IF (Person is Child < 18) AND (Head has insurance) -> YES
    # ELSE -> NO
    
    df["imputed_child"] = ((df[VAR_AGE] < 18) & (df["head_has_ins"] == 1))
    df["final_coverage"] = (df["raw_insurance"] | df["imputed_child"]).astype(int)
    
    print(f"Base Coverage (sh21): {df['raw_insurance'].mean():.2%}")
    print(f"New Coverage (Imputed): {df['final_coverage'].mean():.2%}")

    # --- AGGREGATION ---
    data_rows = {}

    # A. Districts (Kigali)
    df_kigali = df[df[VAR_REG] == 1]
    for dist_code, dist_name in DIST_MAP.items():
        subset = df_kigali[df_kigali[VAR_DIST] == dist_code]
        data_rows[dist_name] = get_triplet(subset)

    # B. Kigali City
    data_rows["Kigali City"] = get_triplet(df_kigali)

    # C. Rwanda
    data_rows["Rwanda"] = get_triplet(df)

    # Convert to DataFrame
    final_df = pd.DataFrame.from_dict(data_rows, orient='index')
    final_df = final_df[["Men", "Women", "Total"]] 
    
    print("\n--- Health Insurance (With Child Imputation) ---")
    print(final_df)
    final_df.to_csv("Insurance_Imputed.csv")

    # ------------------------------------------------------------------
    # 4. PLOTTING
    # ------------------------------------------------------------------
    ax = final_df.plot(kind="bar", color=COLORS, figsize=(14, 7), 
                       width=0.75, edgecolor="white")

    plt.title(f"Health Insurance Coverage\n(De jure, Imputed for Children <18)", 
              fontsize=16, fontweight="bold")
    plt.xticks(rotation=0, fontsize=11)
    plt.grid(axis="y", ls="--", alpha=0.3)
    ax.yaxis.set_visible(False)
    for s in ["top", "right", "left"]: ax.spines[s].set_visible(False)

    plt.legend(ncol=3, loc="upper center", bbox_to_anchor=(0.5, -0.08), 
               frameon=False, fontsize=12)

    for c in ax.containers:
        for v in c:
            height = v.get_height()
            label = f"{int(height)}" if height > 0 else ""
            ax.text(v.get_x() + v.get_width() / 2, height + 1, label,
                    ha='center', va='bottom', fontsize=11, fontweight="bold", color="black")

    plt.tight_layout()
    plt.savefig("Insurance_Imputed.png", dpi=300)
    plt.close()
    print("✅ Analysis Complete.")

Loading Data...


/var/folders/jr/x6fm65cx0hlblk09pm9kft3h0000gn/T/ipykernel_17277/4086532456.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["w"] = df[VAR_WT] / 1000000.0
/var/folders/jr/x6fm65cx0hlblk09pm9kft3h0000gn/T/ipykernel_17277/4086532456.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["raw_insurance"] = (df[VAR_INSURANCE] == 1).astype(int)
/var/folders/jr/x6fm65cx0hlblk09pm9kft3h0000gn/T/ipykernel_17277/4086532456.py:101: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.in

Applying Household Imputation Logic...


/var/folders/jr/x6fm65cx0hlblk09pm9kft3h0000gn/T/ipykernel_17277/4086532456.py:102: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["final_coverage"] = (df["raw_insurance"] | df["imputed_child"]).astype(int)


Base Coverage (sh21): 81.90%
New Coverage (Imputed): 84.25%

--- Health Insurance (With Child Imputation) ---
             Men  Women  Total
Nyarugenge    78     80     79
Gasabo        83     81     82
Kicukiro      86     87     87
Kigali City   83     83     83
Rwanda        83     84     84
✅ Analysis Complete.
